# Particle Fluid

Modeling fluid motion by simulating a set of particles. 

Credit to Professor Bo Zhu, on whose Dartmouth COSC89 course notes the following code is based

In [99]:
import taichi as ti
import taichi.math as tm
import math

ti.init()

[Taichi] Starting on arch=x64


In [100]:
### Global variables ###
dt = 2e-3
num_particles = 10
substeps = 10
pressure_coef = 1e1
density_0 = 10.0
viscosity_coef = 1e1
gravity = tm.vec2(0, -0.5)
max_num_particles_per_cell = 100
max_num_neighbors = 100
screen_size = 800
ks = 2e2
kd = 0.5e1

particle_radius = 5
h_raw = 15 # in terms of pts
h = h_raw/ screen_size # fraction of the screen that it represents
bowl_r = 500
c = tm.vec2(0.5, 0.7) # center of the bowl

num_voxels_per_side = int(screen_size / h_raw) # assuming a square screen
num_voxels = int(pow(screen_size / h_raw, 2))

pos = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
vel = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
force = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
r = ti.field(dtype=ti.f32, shape=num_particles)
m = ti.field(dtype=ti.f32, shape=num_particles)
density = ti.field(dtype=ti.f32, shape=num_particles)
pressure = ti.field(dtype=ti.f32, shape=num_particles)
viscosity = ti.field(dtype=ti.f32, shape=num_particles)
neighbors = ti.field(dtype=ti.i32, shape=(num_particles, max_num_neighbors)) # dictionary mapping a an int (particle idx) -> list of indices of neighbors
voxels = ti.field(dtype=ti.i32, shape=(num_voxels, max_num_particles_per_cell)) 
# voxels = ti.field(ti.i32) # dictionary mapping a voxel coordinates -> list of indices of points in that box
# ti.root.dense(ti.ij, (num_voxels, max_num_particles_per_cell)).place(voxels) # num_voxels x max_num_particles_per_cell

particle_num_neighbors = ti.field(dtype=ti.i32, shape=num_particles)
grid_num_particles = ti.field(dtype=ti.i32, shape=num_voxels)

In [101]:
### Spiky Kernel Functions ###
@ti.func
def w_spiky(xji) -> ti.f32: # change this to pass in a vector?
    valToReturn = float(0)
    r = xji.norm()
    if (r > 0 and r <= h):
        valToReturn = 15.0 / (math.pi * ti.pow(h, 6)) * ti.pow(h - r, 3)
    return valToReturn

@ti.func
def gradient_w_spiky(v) -> tm.vec2: 
    valToReturn = tm.vec2(0,0)
    r = v.norm() 
    if (r > 0 and r <= h):
        valToReturn = -45.0 / (math.pi * ti.pow(h, 6)) * ti.pow (h - r, 2) * v / r
    return valToReturn

@ti.func
def laplacian_w_spiky(v) -> ti.f32: # if v the velocity or the displacement vector
    valToReturn = 0.0
    r = v.norm() 
    if (r > 0 and r <= h):
        valToReturn = -90.0 / (math.pi * ti.pow(h, 6)) * (h - r) * (h - 2*r) / r
    return valToReturn


In [102]:
## Viscosity Kernel Functions ###
@ti.func
def w_vis(xji) -> ti.f32: # do I need this
    valToReturn = 0.0
    r = xji.norm()
    if (r > 0 and r <= h):
        # valToReturn = 15.0 / (2 * math.pi * ti.pow(h, 3)) * (-r / (2 * ti.pow(h, 3)) + ti.pow(r, 2)/ti.pow(h, 2) + h / (2 * r) - 1)
        valToReturn = 15.0 / (2 * math.pi * ti.pow(h,3)) * ((- ti.pow(r, 3) / (2 * ti.pow(h, 3)) + ti.pow(r, 2) / ti.pow(h, 2) + h / (2 * r) - 1))
    return valToReturn

@ti.func
def gradient_w_vis(v) -> tm.vec2: # do I need this
    valToReturn = tm.vec2(0,0)
    r = v.norm()
    if (r > 0 and r <= h):
        valToReturn = 15.0 / (2 * math.pi * ti.pow(h, 3)) * (-3 * r / (2 * ti.pow(h, 3)) + 2 / ti.pow(h, 2) - h/ (2 * ti.pow(r, 3))) * v
    return valToReturn

@ti.func
def laplacian_w_vis(v) -> ti.f32:
    valToReturn = 0.0
    r = v.norm()
    if (r > 0 and r <= h):
        valToReturn = 45.0 / (math.pi * ti.pow(h, 6)) * (h - r)
    return valToReturn


In [140]:
### Spatial Hashing Functions ###

# takes a point index and returns the voxel number that it belongs to
@ti.func
def get_voxel_num(idx:ti.i32) -> ti.i32:
    voxelX = int(pos[idx][0] / h)
    voxelY = int(pos[idx][1] / h)
    voxel_num = voxelY * num_voxels_per_side + voxelX
    return voxel_num

# takes the voxel X and voxelY and returns the voxel number
@ti.func
def get_voxel_num(voxelX:ti.i32, voxelY:ti.i32) -> ti.i32:
    return int(voxelY * num_voxels_per_side + voxelX)

@ti.func
def get_voxel_x(idx:ti.i32) -> ti.i32:
    return int(pos[idx][0] / h)

@ti.func
def get_voxel_y(idx:ti.i32) -> ti.i32:
    return int(pos[idx][1] / h)

# reset all voxel index lists to be -1
@ti.func
def clear_voxels():
    for i in range(num_voxels):
        grid_num_particles[i] = 0
        for j in range(max_num_particles_per_cell):
            voxels[i, j] = -1

# @param - index of the point being added to voxels
@ti.func
def add_point(idx:ti.i32):
    voxel_num = get_voxel_num(idx)
    
    count = grid_num_particles
    voxels[voxel_num][count] = idx
    grid_num_particles += 1


# Redo the spatial hashing for each timestep
@ti.kernel
def update_voxels():
    clear_voxels()
    for idx in range(num_particles):
        add_point(idx)

In [141]:
### Simulator Functions ###
@ti.func
def clear_force():
    for idx in range(num_particles):
        force[idx] = tm.vec2(0, 0)

# reset all neighbors index lists to be -1
@ti.func
def clear_neighbors():
    for i in range(num_particles):
        for j in range(max_num_neighbors):
            neighbors[i, j] = -1
            
# add all particles in voxel with coords (voxelX, voxelY) to the neighbors list of point idx
@ti.func
def update_neighbors_helper(idx:ti.i32, voxelX:ti.i32, voxelY:ti.i32):
    voxel_num = get_voxel_num(voxelX, voxelY)
    count = particle_num_neighbors[idx]
    voxCounter = 0
    while voxels[voxel_num, voxCounter] != -1 and count < max_num_neighbors:
        neighbors[idx, count] = voxels[voxel_num, voxCounter] 
        particle_num_neighbors[idx] += 1
        count = count+1
        voxCounter = voxCounter+1
        
@ti.func
def update_neighbors():
    clear_neighbors()
    for idx in range(num_particles):
        # look at 9 cells surrounding
        voxelX = get_voxel_x(idx)
        voxelY = get_voxel_y(idx)
        # upper row
        if voxelY > 0:
            if voxelX > 0: # upper left
                update_neighbors_helper(idx, voxelX-1, voxelY+1)
            # up
            update_neighbors_helper(idx, voxelX, voxelY+1)
            if voxelX < num_voxels_per_side: # upper right
                update_neighbors_helper(idx, voxelX+1, voxelY+1)
        # middle row
        if voxelX > 0: # left
            update_neighbors_helper(idx, voxelX-1, voxelY)
        # center
        update_neighbors_helper(idx, voxelX, voxelY)
        if voxelX < num_voxels_per_side: # right
            update_neighbors_helper(idx, voxelX+1, voxelY)
        
        # lower row
        if voxelY < num_voxels_per_side:
            if voxelX > 0: # lower left
                update_neighbors_helper(idx, voxelX-1, voxelY-1)
            # down
            update_neighbors_helper(idx, voxelX, voxelY-1)
            if voxelX < num_voxels_per_side: # lower right
                update_neighbors_helper(idx, voxelX+1, voxelY-1)

@ti.func
def update_density():
    for idx in range(num_particles):
        d = 0.0
        for nbr in range(particle_num_neighbors[idx]):
            nbr_idx = neighbors[idx, nbr]
            xji = pos[idx] - pos[nbr_idx]
            d += m[nbr_idx] * w_spiky(xji)
        density[idx] = d
#         density[idx] = 1

@ti.func
def update_pressure():
    for idx in range(num_particles):
        pressure[idx] = pressure_coef * (density[idx] - density_0)

@ti.func
def update_pressure_force():
    for idx in range(num_particles):
        currForce = tm.vec2(0,0)
        for nbr in range(particle_num_neighbors[idx]):
            nbr_idx = neighbors[idx, nbr]
            xji = pos[idx] - pos[nbr_idx]
            currForce += -(pressure[idx] + pressure[nbr_idx]) * 0.5 * m[idx] / density[idx] * gradient_w_spiky(xji)
        force[idx] += currForce

@ti.func
def update_viscosity_force():
    for idx in range(num_particles):
        currForce = tm.vec2(0,0)
        for nbr in range(particle_num_neighbors[idx]):
            nbr_idx = neighbors[idx, nbr]
            xji = pos[idx] - pos[nbr_idx]
            currForce += viscosity_coef * (vel[nbr_idx] - vel[idx]) * m[idx] / density[idx] * laplacian_w_vis(xji)
        force[idx] += currForce

@ti.func
def update_body_force():
    for idx in range(num_particles):
        force[idx] += density[idx] * gravity

@ti.func
def update_boundary_collision_force():
    for i in range(num_particles):
        # boundary check with the bowl
        phi = bowl_r - ((pos[i] - c).norm() * screen_size) - r[i]
        neg_grad = (pos[i] - c).normalized()
        if phi < 0: # outside of the bowl
            force[i] += ks * (phi) * neg_grad
            force[i] += kd * (ti.Vector([0, 0]) - vel[i]).dot(neg_grad) * neg_grad

@ti.kernel
def substep():
    clear_force()
    update_neighbors()    
    update_density()
#     update_pressure()
#     update_pressure_force()
#     update_viscosity_force()
    update_body_force()
    update_boundary_collision_force()
    # update position and velocity
    for idx in range(num_particles):
        vel[idx] = vel[idx] + dt * force[idx] / m[idx]
        pos[idx] = pos[idx] + dt * vel[idx]

In [142]:
### Setup ###
@ti.kernel
def init_particles():
    sqrt = int(ti.sqrt(num_particles))
    disp = 0.5/sqrt
    for idx in range(num_particles):
        r[idx] = particle_radius
        vel[idx] = tm.vec2(0, 0)
        force[idx] = tm.vec2(0, 0)
        m[idx] = 1.0
        pos[idx] = tm.vec2(0.25 + disp * (int(idx/ sqrt)), 0.25 + disp * (idx % sqrt))   

In [144]:
### Driver ###
def main():
    gui = ti.GUI('Particle Fluid', (screen_size, screen_size))
        
    init_particles()
    substep()
    print(num_voxels_per_side)
#    print(neighbors)
        
    while gui.running:
            
#         Move stuff
        for step in range(substeps):
            substep()
             
        # Draw particles
        X = pos.to_numpy()
        for i in range(num_particles):
            gui.circle(pos=X[i], radius = r[i])
        
        gui.show()
    
if __name__ == '__main__':
    main()

53


### To do

Keep another vector called partcle_num_neighbors to keep track of how many neighbors the given particle has at any point?